In [2]:
# ##### Create a KG
import rdflib
from rdflib import URIRef
from ampligraph.evaluation import train_test_split_no_unseen
from rdflib.namespace import Namespace, NamespaceManager
from rdflib.namespace import RDF, RDFS
from collections import ChainMap
from rdflib import Dataset
from rdflib import Graph
import networkx as nx
import pandas as pd
import numpy as np
import pickle
import json
import uuid

In [2]:
with open('/work/ujaimini/CLEVRER/ClevrerHuman_Dataset/CegWithNoCycles_1.txt', 'r') as f:
    cegWithNoCycles = json.load(f)

with open('/work/ujaimini/CLEVRER/ClevrerHuman_Dataset/RawData/VerbObject_1.json', 'r') as f:
        verbObject = json.load(f)

file = open('/work/ujaimini/CLEVRER/ClevrerHuman_Dataset/RawData/valid_ceg_data_May12.p', 'rb')  
validData = pickle.load(file)
file.close()

file = open('/work/ujaimini/CLEVRER/ClevrerHuman_Dataset/RawData/train_ceg_data_May12.p', 'rb')  
trainData = pickle.load(file)
file.close()

data = ChainMap(trainData, validData)

In [3]:
# For the mediator triples, look for depth 2 or 3 for creating the dataset. 
# Consider the Markov split at Depth 2

# Read the list of CEG ids with given Depth
file = open('/work/ujaimini/CLEVRER/CausalDisco/CausalCLEVERERHumanKG_CEGBasedSplit/trainVideoIdCEGList.pkl', 'rb')  
trainCEG = pickle.load(file)
file.close()

file = open('/work/ujaimini/CLEVRER/CausalDisco/CausalCLEVERERHumanKG_CEGBasedSplit/testVideoIdCEGList.pkl', 'rb')  
testCEG = pickle.load(file)
file.close()

In [4]:
fileHyper=open("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph.txt","w")
fileTriples=open("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples.txt","w")

In [5]:
# file=open("Triple"+"KG.txt","w")

clevrerHumans = Namespace("http://semantic.bosch.com/CausalClevrerHumans/v00/")
clevrerHumansData = Namespace("http://semantic.bosch.com/CausalClevrerHumans/v00/data/")
clevrer = Namespace("http://semantic.bosch.com/CLEVRER/")
causal = Namespace("http://semantic.bosch.com/causal/v00/")
scene = Namespace("http://semantic.bosch.com/scene/v02/")
ssn = Namespace("http://www.w3.org/ns/ssn/")

# g = Graph()
# g = Dataset()
# namespace_manager = NamespaceManager(Graph())
# namespace_manager.bind('', clevrerHumansData, override=False)
# namespace_manager.bind("causal", causal, override=True)
# namespace_manager.bind("CCH", clevrerHumans, override=True)
# namespace_manager.bind("scene", scene, override=True)
# namespace_manager.bind("ssn", ssn, override=True)
# g.namespace_manager = namespace_manager


#### Read the CEG to consider for mediators


In [6]:
# for id in cegWithNoCycles:
#     id = '00053'

#     print(id)
#     ceg_full = data[id].get('CEG_full')
# #     print(type(ceg_full))
#     threshold = 1
#     # filter out all edges above threshold and grab id's
#     long_edges = list(filter(lambda e: e[2] == threshold, (e for e in ceg_full.edges.data('weight'))))
#     le_ids = list(e[:2] for e in long_edges)

#     # remove filtered edges from graph G
#     ceg_full.remove_edges_from(le_ids)

#     long_edges = list(filter(lambda e: e[2] == threshold, (e for e in ceg_full.edges.data('width'))))
#     le_ids = list(e[:2] for e in long_edges)
#     # remove filtered edges from graph G
#     ceg_full.remove_edges_from(le_ids)
    
#     rootList = list()
#     [rootList.append(n) for n,d in ceg_full.in_degree() if d==0]
    
#     print("RootList: ",rootList)
    
#     dfs = list()
#     for root in rootList:
#         print("Root: ", root, "\n")
#         l = (list(nx.dfs_edges(ceg_full, source=root, depth_limit=2))) #dfs_tree
#         print(l,"\n")
#         for i in range(len(l)):
# #             print(l[i])
#             if l[i][0]==l[i-1][1]:
#                 print("The mediator l[i][0]: ", l[i][0])
#                 print([l[i-1][0], l[i-1][1], l[i][1]],"\n")
# #                 break
# #         break
# #         dfs.append(list(nx.dfs_edges(ceg_full, source=root, depth_limit=2)))
# #         trainSetNodes.add(root)
# #     print(list(l.edges()))
    
# #     dfs=sorted(dfs, key=len, reverse=True)
# #     print(dfs)
#     ids = id
#     break

In [7]:
# # Add for the Shape, color and material property to the graph
# # Shape: Cube, Sphere, Ball, 
# # Get unique shape, color, and material
shapeSet = set()
colorSet = set()
materialSet = set()

for ids in cegWithNoCycles:
# for ids in ['00053']:
    cegdata = data[ids].get('CEG_full')
    for k in cegdata.nodes(): 
        if 'Color' in verbObject[ids][k]['Object'][0].keys():
            colorSet.add(verbObject[ids][k]['Object'][0]['Color'])
        if 'Material' in verbObject[ids][k]['Object'][0].keys():    
            materialSet.add(verbObject[ids][k]['Object'][0]['Material'])
        if 'Shape' in verbObject[ids][k]['Object'][0].keys():    
            shapeSet.add(verbObject[ids][k]['Object'][0]['Shape'])

In [8]:
shape = dict()
color = dict()
material = dict()

for c in colorSet:
    colorUUID = URIRef(clevrerHumansData+str(uuid.uuid4()))   
    color[c] = colorUUID
#     g.add((colorUUID, RDF.type, URIRef(clevrerHumans+"Color"))) 
#     g.add((colorUUID, RDFS.label, rdflib.term.Literal(c))) 
    fileTriples.writelines([colorUUID,",", RDF.type,",", URIRef(clevrerHumans+"Color"),".","\n"])
    fileTriples.writelines([colorUUID,",", RDFS.label,",", rdflib.term.Literal(c),".","\n"])
    fileHyper.writelines([colorUUID,",", RDF.type,",", URIRef(clevrerHumans+"Color"),".","\n"])
    fileHyper.writelines([colorUUID,",", RDFS.label,",", rdflib.term.Literal(c),".","\n"])

for s in shapeSet:
    shapeUUID = URIRef(clevrerHumansData+str(uuid.uuid4()))  
    shape[s] = shapeUUID
#     g.add((shapeUUID, RDF.type, URIRef(clevrerHumans+"Shape"))) 
#     g.add((shapeUUID, RDFS.label, rdflib.term.Literal(s))) 
    fileTriples.writelines([shapeUUID,",", RDF.type,",", URIRef(clevrerHumans+"Shape"),".","\n"])
    fileTriples.writelines([shapeUUID,",", RDFS.label,",", rdflib.term.Literal(s),".","\n"])
    fileHyper.writelines([shapeUUID,",", RDF.type,",", URIRef(clevrerHumans+"Shape"),".","\n"])
    fileHyper.writelines([shapeUUID,",", RDFS.label,",", rdflib.term.Literal(s),".","\n"])

for m in materialSet:
    materialUUID = URIRef(clevrerHumansData+str(uuid.uuid4()))   
    material[m] = materialUUID
#     g.add((materialUUID, RDF.type, URIRef(clevrerHumans+"Material")))  
#     g.add((materialUUID, RDFS.label, rdflib.term.Literal(m))) 
    fileTriples.writelines([materialUUID,",", RDF.type,",", URIRef(clevrerHumans+"Material"),".","\n"])
    fileTriples.writelines([materialUUID,",", RDFS.label,",", rdflib.term.Literal(m),".","\n"])
    fileHyper.writelines([materialUUID,",", RDF.type,",", URIRef(clevrerHumans+"Material"),".","\n"])
    fileHyper.writelines([materialUUID,",", RDFS.label,",", rdflib.term.Literal(m),".","\n"])


In [9]:
objectParticipant = dict()
for c in color:
    for s in shape:
        objectParticipantUUID = URIRef(clevrerHumansData+str(uuid.uuid4()))             
        objectParticipant[c+"_"+s] = objectParticipantUUID
#         g.add(((objectParticipantUUID), RDF.type, scene.Object)) 
#         g.add(((objectParticipantUUID), RDFS.label, rdflib.term.Literal(c+" "+s))) 
        fileTriples.writelines([objectParticipantUUID,",", RDF.type,",", scene.Object,".","\n"])
        fileTriples.writelines([objectParticipantUUID,",", RDFS.label,",", rdflib.term.Literal(c+" "+s),".","\n"])
        fileHyper.writelines([objectParticipantUUID,",", RDF.type,",", scene.Object,".","\n"])
        fileHyper.writelines([objectParticipantUUID,",", RDFS.label,",", rdflib.term.Literal(c+" "+s),".","\n"])

        
#         g.add(((objectParticipantUUID), ssn.hasProperty, color[c]))
#         g.add(((objectParticipantUUID), ssn.hasProperty, shape[s]))

        fileTriples.writelines([objectParticipantUUID,",", ssn.hasProperty,",", color[c],".","\n"])
        fileTriples.writelines([objectParticipantUUID,",", ssn.hasProperty,",", shape[s],".","\n"])
        fileHyper.writelines([objectParticipantUUID,",", ssn.hasProperty,",", color[c],".","\n"])
        fileHyper.writelines([objectParticipantUUID,",", ssn.hasProperty,",", shape[s],".","\n"])

In [10]:
for c in color:
    for s in shape:
        for m in material:
            if c not in ['silver', 'gold']:
                objectParticipantUUID = URIRef(clevrerHumansData+str(uuid.uuid4()))             
                objectParticipant[c+"_"+m+"_"+s] = objectParticipantUUID
#                 g.add(((objectParticipantUUID), RDF.type, scene.Object)) 
#                 g.add(((objectParticipantUUID), RDFS.label, rdflib.term.Literal(c+" "+m+" "+s))) 
                fileTriples.writelines([objectParticipantUUID,",", RDF.type,",", scene.Object,".","\n"])
                fileTriples.writelines([objectParticipantUUID,",", RDFS.label,",", rdflib.term.Literal(c+" "+m+" "+s),".","\n"])
                fileHyper.writelines([objectParticipantUUID,",", RDF.type,",", scene.Object,".","\n"])
                fileHyper.writelines([objectParticipantUUID,",", RDFS.label,",", rdflib.term.Literal(c+" "+m+" "+s),".","\n"])

#                 g.add(((objectParticipantUUID), ssn.hasProperty, color[c]))
#                 g.add(((objectParticipantUUID), ssn.hasProperty, shape[s]))
#                 g.add(((objectParticipantUUID), ssn.hasProperty, material[m]))
                
                fileTriples.writelines([objectParticipantUUID,",", ssn.hasProperty,",", color[c],".","\n"])
                fileTriples.writelines([objectParticipantUUID,",", ssn.hasProperty,",", shape[s],".","\n"])
                fileTriples.writelines([objectParticipantUUID,",", ssn.hasProperty,",", material[m],".","\n"])
                fileHyper.writelines([objectParticipantUUID,",", ssn.hasProperty,",", color[c],".","\n"])
                fileHyper.writelines([objectParticipantUUID,",", ssn.hasProperty,",", shape[s],".","\n"])
                fileHyper.writelines([objectParticipantUUID,",", ssn.hasProperty,",", material[m],".","\n"])


In [11]:
for ids in cegWithNoCycles:
# for ids in ['00053']:
# IDS =['10016', '10038', '10063']
# for ids in IDS:
#     g.add((URIRef(clevrerHumansData+(ids)), RDF.type, scene.Scene))
    fileTriples.writelines([URIRef(clevrerHumansData+(ids)),",", RDF.type,",", scene.Scene,".","\n"])
    fileHyper.writelines([URIRef(clevrerHumansData+(ids)),",", RDF.type,",", scene.Scene,".","\n"])

    
    ceg_data = data[ids]
    ceg_full=ceg_data.get('CEG_full')
    
    threshold = 1
    # filter out all edges above threshold and grab id's
    long_edges = list(filter(lambda e: e[2] == threshold, (e for e in ceg_full.edges.data('weight'))))
    le_ids = list(e[:2] for e in long_edges)

    # remove filtered edges from graph G
    ceg_full.remove_edges_from(le_ids)

    long_edges = list(filter(lambda e: e[2] == threshold, (e for e in ceg_full.edges.data('width'))))
    le_ids = list(e[:2] for e in long_edges)
    # remove filtered edges from graph G
    ceg_full.remove_edges_from(le_ids)
    
    nodeDict = dict()
    for n in ceg_full.nodes():
        nodeDict[n] = str(uuid.uuid4())
        
    for k in ceg_full.adj.keys(): 
        cegSubjectNode = URIRef(str(k)) 
        subjectUUID = URIRef(str(nodeDict[k]))             
        for nodes in ceg_full.adj[k]:
            if ceg_full.get_edge_data(k, nodes)['weight'] > 1:
                cegObjectNode =  URIRef(str(nodes)) #URIRef(str(uuid.uuid4()))
                objectUUID =  URIRef(str(nodeDict[nodes]))
                
#                 g.add((URIRef(clevrerHumansData+(ids)), scene.includes, URIRef(clevrerHumansData+subjectUUID))) 
#                 g.add((URIRef(clevrerHumansData+subjectUUID), RDFS.label, rdflib.term.Literal(cegSubjectNode))) 
                fileTriples.writelines([URIRef(clevrerHumansData+(ids)),",", scene.includes,",", URIRef(clevrerHumansData+subjectUUID),".","\n"])
                fileTriples.writelines([URIRef(clevrerHumansData+subjectUUID),",", RDFS.label,",", rdflib.term.Literal(cegSubjectNode),".","\n"])
                fileHyper.writelines([URIRef(clevrerHumansData+(ids)),",", scene.includes,",", URIRef(clevrerHumansData+subjectUUID),".","\n"])
                fileHyper.writelines([URIRef(clevrerHumansData+subjectUUID),",", RDFS.label,",", rdflib.term.Literal(cegSubjectNode),".","\n"])

#                 g.add((URIRef(clevrerHumansData+(ids)), scene.includes, URIRef(clevrerHumansData+objectUUID))) 
#                 g.add((URIRef(clevrerHumansData+objectUUID), RDFS.label, rdflib.term.Literal(cegObjectNode))) 
                fileTriples.writelines([URIRef(clevrerHumansData+(ids)),",", scene.includes,",", URIRef(clevrerHumansData+objectUUID),".","\n"])
                fileTriples.writelines([URIRef(clevrerHumansData+objectUUID),",", RDFS.label,",", rdflib.term.Literal(cegObjectNode),".","\n"])
                fileHyper.writelines([URIRef(clevrerHumansData+(ids)),",", scene.includes,",", URIRef(clevrerHumansData+objectUUID),".","\n"])
                fileHyper.writelines([URIRef(clevrerHumansData+objectUUID),",", RDFS.label,",", rdflib.term.Literal(cegObjectNode),".","\n"])
               
                for objects in verbObject[ids][k]['Object']:
                    if objects:
                        if 'Material' in objects.keys():
#                             print(ids,k,objects, objects.keys(),objects['Material'])
#                             print(objects['Color'], objects['Material'], objects['Shape'])
                            uid = objectParticipant[objects['Color']+"_"+objects['Material']+"_"+objects['Shape']]
                            
#                             g.add((URIRef(clevrerHumansData+(ids)), scene.includes, uid))
                            fileTriples.writelines([URIRef(clevrerHumansData+(ids)),",", scene.includes,",", uid,".","\n"])
                            fileHyper.writelines([URIRef(clevrerHumansData+(ids)),",", scene.includes,",", uid,".","\n"])
                            
#                             g.add((URIRef(clevrerHumansData+subjectUUID), scene.hasParticipant, uid))
#                             g.add((uid, scene.isParticipantIn, URIRef(clevrerHumansData+subjectUUID)))
                            fileTriples.writelines([URIRef(clevrerHumansData+subjectUUID),",", scene.hasParticipant,",", uid,".","\n"])
                            fileTriples.writelines([uid,",", scene.isParticipantIn,",", URIRef(clevrerHumansData+subjectUUID),".","\n"])
                            fileHyper.writelines([URIRef(clevrerHumansData+subjectUUID),",", scene.hasParticipant,",", uid,".","\n"])
                            fileHyper.writelines([uid,",", scene.isParticipantIn,",", URIRef(clevrerHumansData+subjectUUID),".","\n"])
  
                        else:
                            uid = objectParticipant[objects['Color']+"_"+objects['Shape']]
                            
#                             g.add((URIRef(clevrerHumansData+(ids)), scene.includes, uid))
                            fileTriples.writelines([URIRef(clevrerHumansData+(ids)),",", scene.includes,",", uid,".","\n"])
                            fileHyper.writelines([URIRef(clevrerHumansData+(ids)),",", scene.includes,",", uid,".","\n"])
            
#                             g.add((URIRef(clevrerHumansData+subjectUUID), scene.hasParticipant, uid))
#                             g.add((uid, scene.isParticipantIn, URIRef(clevrerHumansData+subjectUUID)))
                            fileTriples.writelines([URIRef(clevrerHumansData+subjectUUID),",", scene.hasParticipant,",", uid,".","\n"])
                            fileTriples.writelines([uid,",", scene.isParticipantIn,",", URIRef(clevrerHumansData+subjectUUID),".","\n"])
                            fileHyper.writelines([URIRef(clevrerHumansData+subjectUUID),",", scene.hasParticipant,",", uid,".","\n"])
                            fileHyper.writelines([uid,",", scene.isParticipantIn,",", URIRef(clevrerHumansData+subjectUUID),".","\n"])
                    
                                
                    
                for objects in verbObject[ids][nodes]['Object']:
                    if objects:
                        if 'Material' in objects.keys():
                            uid = objectParticipant[objects['Color']+"_"+objects['Material']+"_"+objects['Shape']]
                            
#                             g.add((URIRef(clevrerHumansData+(ids)), scene.includes, uid))
                            fileTriples.writelines([URIRef(clevrerHumansData+(ids)),",", scene.includes,",", uid,".","\n"])
                            fileHyper.writelines([URIRef(clevrerHumansData+(ids)),",", scene.includes,",", uid,".","\n"])
 
#                             g.add((URIRef(clevrerHumansData+objectUUID), scene.hasParticipant, uid))
#                             g.add((uid, scene.isParticipantIn, URIRef(clevrerHumansData+objectUUID)))
                            fileTriples.writelines([URIRef(clevrerHumansData+objectUUID),",", scene.hasParticipant,",", uid,".","\n"])
                            fileTriples.writelines([uid,",", scene.isParticipantIn,",", URIRef(clevrerHumansData+objectUUID),".","\n"])
                            fileHyper.writelines([URIRef(clevrerHumansData+objectUUID),",", scene.hasParticipant,",", uid,".","\n"])
                            fileHyper.writelines([uid,",", scene.isParticipantIn,",", URIRef(clevrerHumansData+objectUUID),".","\n"])
                
                        else:
                            uid = objectParticipant[objects['Color']+"_"+objects['Shape']]
                            
#                             g.add((URIRef(clevrerHumansData+(ids)), scene.includes, uid))
                            fileTriples.writelines([URIRef(clevrerHumansData+(ids)),",", scene.includes,",", uid,".","\n"])
                            fileHyper.writelines([URIRef(clevrerHumansData+(ids)),",", scene.includes,",", uid,".","\n"])
                              
#                             g.add((URIRef(clevrerHumansData+objectUUID), scene.hasParticipant, uid))
#                             g.add((uid, scene.isParticipantIn, URIRef(clevrerHumansData+objectUUID)))
                            fileTriples.writelines([URIRef(clevrerHumansData+objectUUID),",", scene.hasParticipant,",", uid,".","\n"])
                            fileTriples.writelines([uid,",", scene.isParticipantIn,",", URIRef(clevrerHumansData+objectUUID),".","\n"])
                            fileHyper.writelines([URIRef(clevrerHumansData+objectUUID),",", scene.hasParticipant,",", uid,".","\n"])
                            fileHyper.writelines([uid,",", scene.isParticipantIn,",", URIRef(clevrerHumansData+objectUUID),".","\n"])

                
                if verbObject[ids][nodes]['Verbs'][0] == 'come':
                    verbObject[ids][nodes]['Verbs'][0] = 'ComeFrom'
                
                if verbObject[ids][nodes]['Verbs'][0] == 'change':
                    verbObject[ids][nodes]['Verbs'][0] = 'changeDirection'
                    
                if verbObject[ids][k]['Verbs'][0] == 'come':
                    verbObject[ids][k]['Verbs'][0] = 'ComeFrom'
                
                if verbObject[ids][k]['Verbs'][0] == 'change':
                    verbObject[ids][k]['Verbs'][0] = 'changeDirection'
                    
#                 g.add((URIRef(clevrerHumansData+subjectUUID), RDF.type, URIRef(clevrerHumans+verbObject[ids][k]['Verbs'][0].capitalize())))
#                 g.add((URIRef(clevrerHumansData+subjectUUID), causal.causesType, URIRef(clevrerHumans+verbObject[ids][nodes]['Verbs'][0].capitalize())))
                fileTriples.writelines([URIRef(clevrerHumansData+subjectUUID),",", RDF.type,",", URIRef(clevrerHumans+verbObject[ids][k]['Verbs'][0].capitalize()),".","\n"])
                fileTriples.writelines([URIRef(clevrerHumansData+subjectUUID),",", causal.causesType,",", URIRef(clevrerHumans+verbObject[ids][nodes]['Verbs'][0].capitalize()),".","\n"])
                fileHyper.writelines([URIRef(clevrerHumansData+subjectUUID),",", RDF.type,",", URIRef(clevrerHumans+verbObject[ids][k]['Verbs'][0].capitalize()),".","\n"])
                fileHyper.writelines([URIRef(clevrerHumansData+subjectUUID),",", causal.causesType,",", URIRef(clevrerHumans+verbObject[ids][nodes]['Verbs'][0].capitalize()),".","\n"])
                
#                 g.add((URIRef(clevrerHumansData+objectUUID), RDF.type, URIRef(clevrerHumans+verbObject[ids][nodes]['Verbs'][0].capitalize())))
#                 g.add((URIRef(clevrerHumansData+objectUUID), causal.causedByType, URIRef(clevrerHumans+verbObject[ids][k]['Verbs'][0].capitalize())))
                fileTriples.writelines([URIRef(clevrerHumansData+objectUUID),",", RDF.type,",", URIRef(clevrerHumans+verbObject[ids][nodes]['Verbs'][0].capitalize()),".","\n"])
                fileTriples.writelines([URIRef(clevrerHumansData+objectUUID),",", causal.causedByType,",", URIRef(clevrerHumans+verbObject[ids][k]['Verbs'][0].capitalize()),".","\n"])
                fileHyper.writelines([URIRef(clevrerHumansData+objectUUID),",", RDF.type,",", URIRef(clevrerHumans+verbObject[ids][nodes]['Verbs'][0].capitalize()),".","\n"])
                fileHyper.writelines([URIRef(clevrerHumansData+objectUUID),",", causal.causedByType,",", URIRef(clevrerHumans+verbObject[ids][k]['Verbs'][0].capitalize()),".","\n"])

                
#                 g.add((URIRef(clevrerHumansData+subjectUUID), causal.causes, URIRef(clevrerHumansData+objectUUID)))
                fileTriples.writelines([URIRef(clevrerHumansData+subjectUUID),",", causal.causes,",", URIRef(clevrerHumansData+objectUUID),".","\n"])
                fileHyper.writelines([URIRef(clevrerHumansData+subjectUUID),",", causal.causes,",", URIRef(clevrerHumansData+objectUUID),".","\n"])

#                 g.add((URIRef(clevrerHumansData+objectUUID), causal.causedBy, URIRef(clevrerHumansData+subjectUUID)))
                fileTriples.writelines([URIRef(clevrerHumansData+objectUUID),",", causal.causedBy,",", URIRef(clevrerHumansData+subjectUUID),".","\n"])
                fileHyper.writelines([URIRef(clevrerHumansData+objectUUID),",", causal.causedBy,",", URIRef(clevrerHumansData+subjectUUID),".","\n"])

                
    rootList = list()
    [rootList.append(n) for n,d in ceg_full.in_degree() if d==0]
    
#     print("\n RootList: ",rootList)
    
    dfs = list()
    for root in rootList:
#         print("\n Root: ", root, "\n")
        l = (list(nx.dfs_edges(ceg_full, source=root, depth_limit=2))) #dfs_tree
#         print(l,"\n")
        for i in range(len(l)):
#             print(l[i])
            if l[i][0]==l[i-1][1]:
        
                if verbObject[ids][l[i][1]]['Verbs'][0] == 'come':
                    verbObject[ids][l[i][1]]['Verbs'][0] = 'ComeFrom'
                
                if verbObject[ids][l[i][1]]['Verbs'][0] == 'change':
                    verbObject[ids][l[i][1]]['Verbs'][0] = 'changeDirection'
                    
                if verbObject[ids][l[i-1][0]]['Verbs'][0] == 'come':
                    verbObject[ids][l[i-1][0]]['Verbs'][0] = 'ComeFrom'
                
                if verbObject[ids][l[i-1][0]]['Verbs'][0] == 'change':
                    verbObject[ids][l[i-1][0]]['Verbs'][0] = 'changeDirection'
                    
#                 print("The mediator l[i][0]: ", l[i][0], nodeDict[l[i][0]])
#                 print([nodeDict[l[i-1][0]], l[i-1][0], nodeDict[l[i][0]], l[i][0], nodeDict[l[i][1]],l[i][1]],"\n")
#                 print(l[i][1]," : ", verbObject[ids][l[i][1]]['Verbs'][0])
                
#                 print(URIRef(clevrerHumansData+nodeDict[l[i-1][0]]), causal.causesType, verbObject[ids][l[i][1]]['Verbs'][0], causal.hasMediator, URIRef(clevrerHumansData+nodeDict[l[i][0]]))
#                 print("\n")
#                 print(URIRef(clevrerHumansData+nodeDict[l[i][1]]), causal.causedByType, verbObject[ids][l[i-1][0]]['Verbs'][0], causal.hasMediator, URIRef(clevrerHumansData+nodeDict[l[i][0]]))
                
#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.causesType,",", URIRef(clevrerHumans+verbObject[ids][l[i][1]]['Verbs'][0].capitalize()),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])
#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.causedByType,",", URIRef(clevrerHumans+verbObject[ids][l[i-1][0]]['Verbs'][0].capitalize()),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])
                    
#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.causesType,",", URIRef(clevrerHumans+verbObject[ids][l[i][1]]['Verbs'][0].capitalize()),",", causal.hasMediator, ",", URIRef(clevrerHumansData+nodeDict[l[i][0]]),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])
#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.causedByType,",", URIRef(clevrerHumans+verbObject[ids][l[i-1][0]]['Verbs'][0].capitalize()),",", causal.hasMediator, ",", URIRef(clevrerHumansData+nodeDict[l[i][0]]),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])

#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.causes,",", URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])
#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.causedBy,",", URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])

#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.causes,",", URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.hasMediator, ",", URIRef(clevrerHumansData+nodeDict[l[i][0]]),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])
#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.causedBy,",", URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.hasMediator, ",", URIRef(clevrerHumansData+nodeDict[l[i][0]]),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])

                #########HasMediatorInstance
#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.causesType,",", URIRef(clevrerHumans+verbObject[ids][l[i][1]]['Verbs'][0].capitalize()),",", causal.hasMediator,",", URIRef(clevrerHumansData+nodeDict[l[i][0]]),".","\n"])
#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.causedByType,",", URIRef(clevrerHumans+verbObject[ids][l[i-1][0]]['Verbs'][0].capitalize()),",", causal.hasMediator,",", URIRef(clevrerHumansData+nodeDict[l[i][0]]),".","\n"])

#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.causes,",", URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.hasMediator,",", URIRef(clevrerHumansData+nodeDict[l[i][0]]),".","\n"])
#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.causedBy,",", URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.hasMediator,",", URIRef(clevrerHumansData+nodeDict[l[i][0]]),".","\n"])


                #########HasMediatorType
                fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.causesType,",", URIRef(clevrerHumans+verbObject[ids][l[i][1]]['Verbs'][0].capitalize()),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])
                fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.causedByType,",", URIRef(clevrerHumans+verbObject[ids][l[i-1][0]]['Verbs'][0].capitalize()),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])

                fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.causes,",", URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])
                fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.causedBy,",", URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])

                #########HasMediatorType
#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.causesType,",", URIRef(clevrerHumans+verbObject[ids][l[i][1]]['Verbs'][0].capitalize()),",", causal.hasMediator, ",", URIRef(clevrerHumansData+nodeDict[l[i][0]]),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])
#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.causedByType,",", URIRef(clevrerHumans+verbObject[ids][l[i-1][0]]['Verbs'][0].capitalize()),",", causal.hasMediator, ",", URIRef(clevrerHumansData+nodeDict[l[i][0]]),",", causal.hasMediatorType,",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])

#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.causes,",", URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.hasMediator, ",", URIRef(clevrerHumansData+nodeDict[l[i][0]]), ",", causal.hasMediatorType, ",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])
#                 fileHyper.writelines([URIRef(clevrerHumansData+nodeDict[l[i][1]]),",", causal.causedBy,",", URIRef(clevrerHumansData+nodeDict[l[i-1][0]]),",", causal.hasMediator, ",", URIRef(clevrerHumansData+nodeDict[l[i][0]]), ",", causal.hasMediatorType, ",", URIRef(clevrerHumansData+verbObject[ids][l[i][0]]['Verbs'][0].capitalize()),".","\n"])

                    
        
                

WARNING - the yellow cube comes from the right does not look like a valid URI, trying to serialize this will break.
WARNING - the yellow cube hit with the blue ball does not look like a valid URI, trying to serialize this will break.
WARNING - the blue ball hit the yellow ball does not look like a valid URI, trying to serialize this will break.
WARNING - the yellow cylinder came from below does not look like a valid URI, trying to serialize this will break.
WARNING - the yellow cube hit with the blue ball does not look like a valid URI, trying to serialize this will break.
WARNING - the blue ball hit the yellow ball does not look like a valid URI, trying to serialize this will break.
WARNING - the blue ball hit the yellow ball does not look like a valid URI, trying to serialize this will break.
WARNING - the yellow ball came from the side does not look like a valid URI, trying to serialize this will break.
WARNING - the blue ball hit the yellow ball does not look like a valid URI, tryi

WARNING - the blue cylinder collided with the yellow cube does not look like a valid URI, trying to serialize this will break.
WARNING - the gray cylinder came from the side does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cylinder collided with the yellow cube does not look like a valid URI, trying to serialize this will break.
WARNING - the purple cylinder tagged the gray cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cylinder tagged the purple cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cylinder collided with the yellow cube does not look like a valid URI, trying to serialize this will break.
WARNING - the purple cylinder came from below does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cylinder collided with the yellow cube does not look like a valid URI, trying to serialize this will break.
WARNING - the

WARNING - the cyan cube moved up a bit does not look like a valid URI, trying to serialize this will break.
WARNING - the cyan cube hits the brown ball does not look like a valid URI, trying to serialize this will break.
WARNING - the cyan cube hits the brown ball does not look like a valid URI, trying to serialize this will break.
WARNING - the cyan cube moved up a bit does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cube moved from the side does not look like a valid URI, trying to serialize this will break.
WARNING - the brown cylinder came from the right does not look like a valid URI, trying to serialize this will break.
WARNING - the yellow cylinder came from the right does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cube moved from the side does not look like a valid URI, trying to serialize this will break.
WARNING - the brown cylinder came from the right does not look like a valid URI, trying to seri

WARNING - the purple ball collides with the purple cube does not look like a valid URI, trying to serialize this will break.
WARNING - the green cube moves does not look like a valid URI, trying to serialize this will break.
WARNING - the cyan cube hit the brown cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the yellow cylinder bumped the brown cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the cyan cube bumped into the yellow cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the cyan cube hit the brown cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the yellow cylinder bumped the brown cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the yellow cylinder bumped the brown cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the cyan cube moved does not look

WARNING - the red cube bumped the red cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the red cube sideswiped the purple cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the red cylinder collides with the green ball does not look like a valid URI, trying to serialize this will break.
WARNING - the green ball is pushed to the west does not look like a valid URI, trying to serialize this will break.
WARNING - the red cube sideswiped the purple cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the red cylinder collides with the green ball does not look like a valid URI, trying to serialize this will break.
WARNING - the green ball is pushed to the west does not look like a valid URI, trying to serialize this will break.
WARNING - the green ball is pushed to the west does not look like a valid URI, trying to serialize this will break.
WARNING - the gray cube hit the red cube

...

WARNING - the gray cube collides with the purple cube does not look like a valid URI, trying to serialize this will break.
WARNING - the silver ball collided with the green cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the gray cube collides with the purple cube does not look like a valid URI, trying to serialize this will break.
WARNING - the cyan cylinder came from below does not look like a valid URI, trying to serialize this will break.
WARNING - the purple cube changes direction does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cube moves does not look like a valid URI, trying to serialize this will break.
WARNING - the purple cube hit the red ball does not look like a valid URI, trying to serialize this will break.
WARNING - the purple cube changes direction does not look like a valid URI, trying to serialize this will break.
WARNING - the red ball collided with the blue cube does not look like a vali

WARNING - the green cube bumped into the silver cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the gray cylinder moved does not look like a valid URI, trying to serialize this will break.
WARNING - the brown cube is pushed to the left does not look like a valid URI, trying to serialize this will break.
WARNING - the brown cube hit the brown ball does not look like a valid URI, trying to serialize this will break.
WARNING - the yellow cylinder came from the right does not look like a valid URI, trying to serialize this will break.
WARNING - the brown cube is pushed to the left does not look like a valid URI, trying to serialize this will break.
WARNING - the yellow cylinder collides with the brown cube does not look like a valid URI, trying to serialize this will break.
WARNING - the brown cube hit the brown ball does not look like a valid URI, trying to serialize this will break.
WARNING - the brown ball came from above does not look like a vali

WARNING - the blue ball hits the yellow ball does not look like a valid URI, trying to serialize this will break.
WARNING - the red ball collided with the green cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the blue ball hits the yellow ball does not look like a valid URI, trying to serialize this will break.
WARNING - the red ball collided with the green cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the red ball moved to the west does not look like a valid URI, trying to serialize this will break.
WARNING - the brown cylinder came from the right does not look like a valid URI, trying to serialize this will break.
WARNING - the purple ball changes direction does not look like a valid URI, trying to serialize this will break.
WARNING - the brown cube hit the brown cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the brown cube collides with the purple ball does not 

WARNING - the blue cube came from above does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cube bumped the blue cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cube hit the green ball does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cylinder bumps the cyan cube does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cube bumped the blue cylinder does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cube hit the green ball does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cylinder bumps the cyan cube does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cube hit the green ball does not look like a valid URI, trying to serialize this will break.
WARNING - the blue cylinder bumps the cyan cube does not look like a valid URI, tryin

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
fileTriples.close()
fileHyper.close()

## Split the dataset into Train, Test, Valid split

In [13]:
# hyper=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph.txt",names=['h','r','t','hasM','m'],header=None,sep=",")
# triple=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples.txt",names=['h','r','t','hasM','m'],header=None,sep=",")

# hyper=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph.txt",names=['h','r','t','hasM1','m1','hasM2','m2'],header=None,sep=",")
# triple=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples.txt",names=['h','r','t','hasM1','m1','hasM2','m2'],header=None,sep=",")

hyper=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph.txt",names=['h','r','t','hasM1','m1'],header=None,sep=",")
triple=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples.txt",names=['h','r','t','hasM1','m1'],header=None,sep=",")



In [14]:
hyper

,h,r,t,hasM1,m1
0,http://semantic.bosch.com/CausalClevrerHumans/...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://semantic.bosch.com/CausalClevrerHumans/...,NaN,NaN
1,http://semantic.bosch.com/CausalClevrerHumans/...,http://www.w3.org/2000/01/rdf-schema#label,yellow.,NaN,NaN
2,http://semantic.bosch.com/CausalClevrerHumans/...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://semantic.bosch.com/CausalClevrerHumans/...,NaN,NaN
3,http://semantic.bosch.com/CausalClevrerHumans/...,http://www.w3.org/2000/01/rdf-schema#label,brown.,NaN,NaN
4,http://semantic.bosch.com/CausalClevrerHumans/...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://semantic.bosch.com/CausalClevrerHumans/...,NaN,NaN
...,...,...,...,...,...
112984,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedBy,http://semantic.bosch.com/CausalClevrerHumans/...,NaN,NaN
112985,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causesType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
112986,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedByType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
112987,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causes,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...


In [15]:
def traintestvalidsplit(hyper, triple,folderName):
    data = hyper.to_numpy()

    # data
    X_train_valid, X_test = train_test_split_no_unseen(data, test_size=0.1)
    X_train, X_valid = train_test_split_no_unseen(X_train_valid, test_size=0.1)

    X_train_df = pd.DataFrame(X_train, index=None, columns=None)
    X_test_df = pd.DataFrame(X_test, index=None, columns=None)
    X_valid_df = pd.DataFrame(X_valid, index=None, columns=None)
    
#     X_train_df.to_csv('/work/ujaimini/StarE_code/StarE/data/clean/CLEVRERHumans/statements/train.txt',columns=None, header=False, index=False)
#     X_test_df.to_csv('/work/ujaimini/StarE_code/StarE/data/clean/CLEVRERHumans/statements/test.txt',columns=None, header=False, index=False)
#     X_valid_df.to_csv('/work/ujaimini/StarE_code/StarE/data/clean/CLEVRERHumans/statements/valid.txt',columns=None, header=False, index=False)

    X_train_df.to_csv('/work/ujaimini/StarE_code/StarE/data/clean/'+folderName+'/statements/train.txt',columns=None, header=False, index=False)
    X_test_df.to_csv('/work/ujaimini/StarE_code/StarE/data/clean/'+folderName+'/statements/test.txt',columns=None, header=False, index=False)
    X_valid_df.to_csv('/work/ujaimini/StarE_code/StarE/data/clean/'+folderName+'/statements/valid.txt',columns=None, header=False, index=False)


    data = triple.to_numpy()

    # data
    X_train_valid, X_test = train_test_split_no_unseen(data, test_size=0.1)
    X_train, X_valid = train_test_split_no_unseen(X_train_valid, test_size=0.1)

    X_train_df = pd.DataFrame(X_train, index=None, columns=None)
    X_test_df = pd.DataFrame(X_test, index=None, columns=None)
    X_valid_df = pd.DataFrame(X_valid, index=None, columns=None)
    
#     X_train_df.to_csv('/work/ujaimini/StarE_code/StarE/data/clean/CLEVRERHumans/triples/train.txt',columns=None, header=False, index=False)
#     X_test_df.to_csv('/work/ujaimini/StarE_code/StarE/data/clean/CLEVRERHumans/triples/test.txt',columns=None, header=False, index=False)
#     X_valid_df.to_csv('/work/ujaimini/StarE_code/StarE/data/clean/CLEVRERHumans/triples/valid.txt',columns=None, header=False, index=False)

    X_train_df.to_csv('/work/ujaimini/StarE_code/StarE/data/clean/'+folderName+'/triples/train.txt',columns=None, header=False, index=False)
    X_test_df.to_csv('/work/ujaimini/StarE_code/StarE/data/clean/'+folderName+'/triples/test.txt',columns=None, header=False, index=False)
    X_valid_df.to_csv('/work/ujaimini/StarE_code/StarE/data/clean/'+folderName+'/triples/valid.txt',columns=None, header=False, index=False)

    return

In [16]:
# traintestvalidsplit(hyper, triple,'CLEVRERHumans')

## Causal Prediction: causesType

In [17]:
# Create KG with just 
# hyper=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph.txt",names=['h','r','t','hasM1','m1','hasM2','m2'],header=None,sep=",")
# triple=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples.txt",names=['h','r','t','hasM1','m1','hasM2','m2'],header=None,sep=",")


hyper=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph.txt",names=['h','r','t','hasM1','m1'],header=None,sep=",")
triple=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples.txt",names=['h','r','t','hasM1','m1'],header=None,sep=",")

In [18]:
# Have causes, causedBy, causesType
CC_Hyper=hyper[hyper['r'].isin(["http://semantic.bosch.com/causal/v00/causes","http://semantic.bosch.com/causal/v00/causedBy","http://semantic.bosch.com/causal/v00/causesType"])]
CC_Triple=triple[triple['r'].isin(["http://semantic.bosch.com/causal/v00/causes","http://semantic.bosch.com/causal/v00/causedBy","http://semantic.bosch.com/causal/v00/causesType"])]


In [19]:
CC_Hyper.to_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph_CausesCausedByCausesType.txt",columns=None, header=False, index=False)
CC_Triple.to_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples_CausesCausedByCausesType.txt",columns=None, header=False, index=False)

In [20]:
traintestvalidsplit(CC_Hyper, CC_Triple, 'CLEVRERHumansPredictionHasMediatorType_C')

In [21]:
# Have causes, causedBy, causesType, Type
CT_Hyper=hyper[hyper['r'].isin(["http://semantic.bosch.com/causal/v00/causes","http://semantic.bosch.com/causal/v00/causedBy","http://semantic.bosch.com/causal/v00/causesType","http://www.w3.org/1999/02/22-rdf-syntax-ns#type"])]
CT_Triple=triple[triple['r'].isin(["http://semantic.bosch.com/causal/v00/causes","http://semantic.bosch.com/causal/v00/causedBy","http://semantic.bosch.com/causal/v00/causesType","http://www.w3.org/1999/02/22-rdf-syntax-ns#type"])]


In [22]:
CT_Hyper.to_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph_CausesCausedByCausesTypeType.txt",columns=None, header=False, index=False)
CT_Triple.to_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples_CausesCausedByCausesTypeType.txt",columns=None, header=False, index=False)

In [23]:
traintestvalidsplit(CT_Hyper, CT_Triple, 'CLEVRERHumansPredictionHasMediatorType_CT')

In [24]:
# Have causes, causedBy, causesType, Type, hasParticipant
CTP_Hyper=hyper[hyper['r'].isin(["http://semantic.bosch.com/causal/v00/causes","http://semantic.bosch.com/causal/v00/causedBy","http://semantic.bosch.com/causal/v00/causesType","http://www.w3.org/1999/02/22-rdf-syntax-ns#type","http://semantic.bosch.com/scene/v02/hasParticipant"])]
CTP_Triple=triple[triple['r'].isin(["http://semantic.bosch.com/causal/v00/causes","http://semantic.bosch.com/causal/v00/causedBy","http://semantic.bosch.com/causal/v00/causesType","http://www.w3.org/1999/02/22-rdf-syntax-ns#type","http://semantic.bosch.com/scene/v02/hasParticipant"])]


In [25]:
CTP_Hyper.to_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph_CausesCausedByCausesTypeTypeHasParticipant.txt",columns=None, header=False, index=False)
CTP_Triple.to_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples_CausesCausedByCausesTypeTypeHasParticipant.txt",columns=None, header=False, index=False)

In [26]:
traintestvalidsplit(CTP_Hyper, CTP_Triple,'CLEVRERHumansPredictionHasMediatorType_CTP')

## Causal Explanation: causedByType

In [27]:
# Create KG with just 
# hyper=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph.txt",names=['h','r','t','hasM1','m1','hasM2','m2'],header=None,sep=",")
# triple=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples.txt",names=['h','r','t','hasM1','m1','hasM2','m2'],header=None,sep=",")


hyper=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph.txt",names=['h','r','t','hasM1','m1'],header=None,sep=",")
triple=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples.txt",names=['h','r','t','hasM1','m1'],header=None,sep=",")

In [28]:
# Have causes, causedBy, causedByType
CC_Hyper=hyper[hyper['r'].isin(["http://semantic.bosch.com/causal/v00/causes","http://semantic.bosch.com/causal/v00/causedBy","http://semantic.bosch.com/causal/v00/causedByType"])]
CC_Triple=triple[triple['r'].isin(["http://semantic.bosch.com/causal/v00/causes","http://semantic.bosch.com/causal/v00/causedBy","http://semantic.bosch.com/causal/v00/causedByType"])]


In [29]:
CC_Hyper.to_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph_CausesCausedByCausedByType.txt",columns=None, header=False, index=False)
CC_Triple.to_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples_CausesCausedByCausedByType.txt",columns=None, header=False, index=False)

In [30]:
traintestvalidsplit(CC_Hyper, CC_Triple, 'CLEVRERHumansExplanationHasMediatorType_C')

In [31]:
# Have causes, causedBy, causedByType, Type
CT_Hyper=hyper[hyper['r'].isin(["http://semantic.bosch.com/causal/v00/causes","http://semantic.bosch.com/causal/v00/causedBy","http://semantic.bosch.com/causal/v00/causedByType","http://www.w3.org/1999/02/22-rdf-syntax-ns#type"])]
CT_Triple=triple[triple['r'].isin(["http://semantic.bosch.com/causal/v00/causes","http://semantic.bosch.com/causal/v00/causedBy","http://semantic.bosch.com/causal/v00/causedByType","http://www.w3.org/1999/02/22-rdf-syntax-ns#type"])]


In [32]:
CT_Hyper.to_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph_CausesCausedByCausedByTypeType.txt",columns=None, header=False, index=False)
CT_Triple.to_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples_CausesCausedByCausedByTypeType.txt",columns=None, header=False, index=False)

In [33]:
traintestvalidsplit(CT_Hyper, CT_Triple, 'CLEVRERHumansExplanationHasMediatorType_CT')

In [34]:
# Have causes, causedBy, causedByType, Type, hasParticipant
CTP_Hyper=hyper[hyper['r'].isin(["http://semantic.bosch.com/causal/v00/causes","http://semantic.bosch.com/causal/v00/causedBy","http://semantic.bosch.com/causal/v00/causedByType","http://www.w3.org/1999/02/22-rdf-syntax-ns#type","http://semantic.bosch.com/scene/v02/hasParticipant"])]
CTP_Triple=triple[triple['r'].isin(["http://semantic.bosch.com/causal/v00/causes","http://semantic.bosch.com/causal/v00/causedBy","http://semantic.bosch.com/causal/v00/causedByType","http://www.w3.org/1999/02/22-rdf-syntax-ns#type","http://semantic.bosch.com/scene/v02/hasParticipant"])]


In [35]:
CTP_Hyper.to_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph_CausesCausedByCausedByTypeTypeHasParticipant.txt",columns=None, header=False, index=False)
CTP_Triple.to_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_Triples_CausesCausedByCausedByTypeTypeHasParticipant.txt",columns=None, header=False, index=False)

In [36]:
traintestvalidsplit(CTP_Hyper, CTP_Triple, 'CLEVRERHumansExplanationHasMediatorType_CTP')

### Count the number of triples, entities, Hyper-relations, relation etc

In [13]:
hyper=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorType/CausalCLEVRERHumans_HyperGraph.txt",names=['h','r','t','hasM1','m1'],header=None,sep=",")


In [14]:
hyper.shape

(112989, 5)

In [21]:
# hyper.drop_duplicates()
hyper['hasM1'].unique()

array([nan, 'http://semantic.bosch.com/causal/v00/hasMediatorType'],
      dtype=object)

In [22]:
# hyper[[hyper['hasM1']=='http://semantic.bosch.com/causal/v00/hasMediator']]
hyper[hyper['hasM1'].isin(["http://semantic.bosch.com/causal/v00/hasMediatorType"])]


,h,r,t,hasM1,m1
592,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causesType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
593,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedByType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
594,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causes,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
595,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedBy,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
876,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causesType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
...,...,...,...,...,...
112869,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedBy,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
112985,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causesType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
112986,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedByType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
112987,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causes,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...


In [23]:
hyper=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplit/CausalCLEVRERHumans_HyperGraph.txt",names=['h','r','t','hasM1','m1'],header=None,sep=",")
hyper[hyper['hasM1'].isin(["http://semantic.bosch.com/causal/v00/hasMediator"])]


,h,r,t,hasM1,m1
592,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causesType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...
593,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedByType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...
594,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causes,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...
595,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedBy,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...
876,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causesType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...
...,...,...,...,...,...
112869,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedBy,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...
112985,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causesType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...
112986,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedByType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...
112987,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causes,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...


In [30]:
hyper['m1'].dropna()

592       http://semantic.bosch.com/CausalClevrerHumans/...
593       http://semantic.bosch.com/CausalClevrerHumans/...
594       http://semantic.bosch.com/CausalClevrerHumans/...
595       http://semantic.bosch.com/CausalClevrerHumans/...
876       http://semantic.bosch.com/CausalClevrerHumans/...
                                ...                        
112869    http://semantic.bosch.com/CausalClevrerHumans/...
112985    http://semantic.bosch.com/CausalClevrerHumans/...
112986    http://semantic.bosch.com/CausalClevrerHumans/...
112987    http://semantic.bosch.com/CausalClevrerHumans/...
112988    http://semantic.bosch.com/CausalClevrerHumans/...
Name: m1, Length: 3424, dtype: object

In [32]:
hyper=pd.read_csv("/work/ujaimini/CLEVRER/CausalDisco_HyperRelation/Data/RandomSplitHasMediatorInstanceType/CausalCLEVRERHumans_HyperGraph.txt",names=['h','r','t','hasM1','m1','hasM2','m2'],header=None,sep=",")
hyper[hyper['hasM1'].isin(["http://semantic.bosch.com/causal/v00/hasMediator"])]


,h,r,t,hasM1,m1,hasM2,m2
592,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causesType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
593,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedByType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
594,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causes,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
595,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedBy,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
876,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causesType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
...,...,...,...,...,...,...,...
112869,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedBy,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
112985,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causesType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
112986,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causedByType,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
112987,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/causes,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediator,http://semantic.bosch.com/CausalClevrerHumans/...,http://semantic.bosch.com/causal/v00/hasMediat...,http://semantic.bosch.com/CausalClevrerHumans/...
